In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
from functions import get_price
from functions import get_title
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

In [11]:
#service to chrome driver
service = Service(executable_path="chromedriver.exe")

#Initializating the driver
driver = webdriver.Chrome(service=service)

driver.get("https://www.netimoveis.com/locacao/minas-gerais/belo-horizonte/apartamento?transacao=locacao&localizacao=BR-MG-belo-horizonte---%2CBR-MG-contagem---&tipo=apartamento&pagina=1")

time.sleep(10)
html = driver.page_source

driver.quit()

soup = BeautifulSoup(html, 'html.parser')

links = soup.find_all('a', class_='link-imovel')

links_list = []

for link in links:
    links_list.append(link.get('href'))

In [12]:
#service to chrome driver
service = Service(executable_path="chromedriver.exe")

#Initializating the driver
driver = webdriver.Chrome(service=service)

driver.get("https://www.netimoveis.com/locacao/minas-gerais/belo-horizonte/apartamento?transacao=locacao&localizacao=BR-MG-belo-horizonte---%2CBR-MG-contagem---&tipo=apartamento&pagina=3")

time.sleep(5)
html = driver.page_source

driver.quit()

soup = BeautifulSoup(html, 'html.parser')

links = soup.find_all('a', class_='link-imovel')

for link in links:
    links_list.append(link.get('href'))

In [13]:
#service to chrome driver
service = Service(executable_path="chromedriver.exe")

#Initializating the driver
driver = webdriver.Chrome(service=service)

driver.get("https://www.netimoveis.com/locacao/minas-gerais/belo-horizonte/apartamento?transacao=locacao&localizacao=BR-MG-belo-horizonte---%2CBR-MG-contagem---&tipo=apartamento&pagina=2")

time.sleep(5)
html = driver.page_source

driver.quit()

soup = BeautifulSoup(html, 'html.parser')

links = soup.find_all('a', class_='link-imovel')

for link in links:
    links_list.append(link.get('href'))

In [14]:
#Function to fetch webpage
def fetch_webpage(url, headers, retries=3, wait=5):
    for i in range(retries):
        response = requests.get("https://www.netimoveis.com"+url, headers=headers)
        if response.status_code == 200:
            return response
        elif response.status_code == 503:
            print(f"Service unavailable. Retrying in {wait} seconds...")
            time.sleep(wait)
    return None

#Function to extract Product Title
def get_title(soup):
    try:
        title = soup.find('h1', attrs={'class':'title text-primary'}).text.strip()
    except:
        title = ""
    return title

#Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find('div', attrs={'class':'detail-value'}).text
    except AttributeError:
        try:
            price = soup.findsoup.find('div', attrs={'class':'detail-value'}).text
        except:
            price = ""
    return price

def get_loc(soup):
    try:
        loc = soup.find('div',attrs={'class':'mb-1 text-gray'}).text.strip()
    except AttributeError:
        try:
            loc = soup.find('div',attrs={'class':'mb-1 text-gray'}).text.strip()
        except:
            loc = ""
    return loc

In [15]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}

In [16]:
dict = {'title':[],'price':[], 'loc':[]}

#loop for extracting product details from each link
for link in links_list:
    new_webpage = requests.get("https://www.netimoveis.com"+link, headers = HEADERS)
    new_soup = BeautifulSoup(new_webpage.content, "html.parser")
    dict['title'].append(get_title(new_soup))
    dict['price'].append(get_price(new_soup))
    dict['loc'].append(get_loc(new_soup))

In [17]:
apartament_df = pd.DataFrame.from_dict(dict)

In [18]:
apartament_df = apartament_df.drop_duplicates()
apartament_df.head()

,title,price,loc
0,Apartamento 1 quarto para alugar 43m² ...,"R$ 1.300,00","Rua dos Comanches, 628, Santa Mônica – Belo Ho..."
2,Apartamento 1 quarto para alugar 43m² ...,"R$ 1.300,00","Rua dos Comanches, 628, Jardim Leblon – Belo H..."
4,Apartamento 2 quartos para alugar 43m² ...,"R$ 1.300,00","Rua dos Comanches, 628, Jardim Leblon – Belo H..."
6,Apartamento 2 quartos para alugar 43m² ...,"R$ 1.300,00","Rua dos Comanches, 628, Santa Mônica – Belo Ho..."
8,Apartamento 4 quartos para alugar 70m² ...,"R$ 1.700,00","Rua São Claret, Silveira – Belo Horizonte"


In [19]:
apartament_df.to_csv("Apartamentos_Net_Imoveis.csv", header = True, index = False)